In [5]:
from helpers.dataset import get_dataset
from src.framework import construct_cim

from helpers.dataset import MUFFIN_TASK, CUSTOM_TASKS, CROSS_TASK_TASKS

results = construct_cim(MUFFIN_TASK, get_dataset(MUFFIN_TASK), "final")

dataset = results['labeled_dataset']

Completed in 6 iterations


In [14]:
from collections import defaultdict
import sys

def display_tutorial_contexts(tutorial, include_keys=None, include_content_types=None):
    """
    print in markdown table format where columns are keys and rows are values
    """
    columns = defaultdict(list)
    
    columns['info_type'] = []

    for piece in tutorial['pieces']:
        if include_content_types is not None and piece['content_type'] not in include_content_types:
            continue
        columns['info_type'].append(piece['content_type'])
        for key, value in piece['labels'].items():
            if include_keys is None or key in include_keys:
                columns[key].append(value[-1])
    
    markdown_table = "| " + " | ".join(columns.keys()) + " |\n"
    markdown_table += "| " + " | ".join(["---"] * len(columns)) + " |\n"
    for i in range(len(columns[list(columns.keys())[0]])):
        markdown_table += "| " + " | ".join([columns[key][i] for key in columns.keys()]) + " |\n"
    markdown_table += "| " + " | ".join(["---"] * len(columns)) + " |\n"
    print(markdown_table)

for i in range(len(dataset)):
    display_tutorial_contexts(dataset[i], ['context_stage'], ['Method'])

| info_type | context_stage |
| --- | --- |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Method | Preparation |
| Meth